In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [4]:
#settings
pd.set_option("display.max_columns", None)

In [5]:
df_o = pd.read_csv("Train.csv")

In [6]:
df_o.head(3)

,ID,country,age_group,travel_with,total_female,total_male,purpose,main_activity,info_source,tour_arrangement,package_transport_int,package_accomodation,package_food,package_transport_tz,package_sightseeing,package_guided_tour,package_insurance,night_mainland,night_zanzibar,payment_mode,first_trip_tz,most_impressing,total_cost
0,tour_0,SWIZERLAND,45-64,Friends/Relatives,1.0,1.0,Leisure and Holidays,Wildlife tourism,"Friends, relatives",Independent,No,No,No,No,No,No,No,13.0,0.0,Cash,No,Friendly People,674602.5
1,tour_10,UNITED KINGDOM,25-44,NaN,1.0,0.0,Leisure and Holidays,Cultural tourism,others,Independent,No,No,No,No,No,No,No,14.0,7.0,Cash,Yes,"Wonderful Country, Landscape, Nature",3214906.5
2,tour_1000,UNITED KINGDOM,25-44,Alone,0.0,1.0,Visiting Friends and Relatives,Cultural tourism,"Friends, relatives",Independent,No,No,No,No,No,No,No,1.0,31.0,Cash,No,Excellent Experience,3315000.0


In [25]:
df = df_o.drop(columns = ['ID'],  axis=1)

In [8]:
print(df.shape)
df.head(10)

(4809, 22)


,country,age_group,travel_with,total_female,total_male,purpose,main_activity,info_source,tour_arrangement,package_transport_int,package_accomodation,package_food,package_transport_tz,package_sightseeing,package_guided_tour,package_insurance,night_mainland,night_zanzibar,payment_mode,first_trip_tz,most_impressing,total_cost
0,SWIZERLAND,45-64,Friends/Relatives,1.0,1.0,Leisure and Holidays,Wildlife tourism,"Friends, relatives",Independent,No,No,No,No,No,No,No,13.0,0.0,Cash,No,Friendly People,674602.5
1,UNITED KINGDOM,25-44,NaN,1.0,0.0,Leisure and Holidays,Cultural tourism,others,Independent,No,No,No,No,No,No,No,14.0,7.0,Cash,Yes,"Wonderful Country, Landscape, Nature",3214906.5
2,UNITED KINGDOM,25-44,Alone,0.0,1.0,Visiting Friends and Relatives,Cultural tourism,"Friends, relatives",Independent,No,No,No,No,No,No,No,1.0,31.0,Cash,No,Excellent Experience,3315000.0
3,UNITED KINGDOM,25-44,Spouse,1.0,1.0,Leisure and Holidays,Wildlife tourism,"Travel, agent, tour operator",Package Tour,No,Yes,Yes,Yes,Yes,Yes,No,11.0,0.0,Cash,Yes,Friendly People,7790250.0
4,CHINA,1-24,NaN,1.0,0.0,Leisure and Holidays,Wildlife tourism,"Travel, agent, tour operator",Independent,No,No,No,No,No,No,No,7.0,4.0,Cash,Yes,No comments,1657500.0
5,UNITED KINGDOM,25-44,NaN,0.0,1.0,Leisure and Holidays,Wildlife tourism,"Travel, agent, tour operator",Package Tour,No,No,No,No,Yes,Yes,No,9.0,3.0,Cash,Yes,Wildlife,120950.0
6,SOUTH AFRICA,45-64,Alone,0.0,1.0,Business,Mountain climbing,"Friends, relatives",Independent,No,No,No,No,No,No,No,9.0,0.0,Cash,Yes,Friendly People,466140.0
7,UNITED STATES OF AMERICA,45-64,Friends/Relatives,1.0,1.0,Leisure and Holidays,Wildlife tourism,"Travel, agent, tour operator",Package Tour,Yes,Yes,Yes,Yes,Yes,Yes,Yes,10.0,3.0,Cash,Yes,Friendly People,3480750.0
8,NIGERIA,25-44,Alone,0.0,1.0,Leisure and Holidays,Cultural tourism,"Travel, agent, tour operator",Independent,No,No,No,No,No,No,No,4.0,0.0,Cash,Yes,NaN,994500.0
9,INDIA,25-44,Alone,1.0,0.0,Business,Wildlife tourism,"Travel, agent, tour operator",Independent,No,No,No,No,No,No,No,5.0,0.0,Credit Card,Yes,Friendly People,2486250.0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                4809 non-null   object 
 1   age_group              4809 non-null   object 
 2   travel_with            3695 non-null   object 
 3   total_female           4806 non-null   float64
 4   total_male             4804 non-null   float64
 5   purpose                4809 non-null   object 
 6   main_activity          4809 non-null   object 
 7   info_source            4809 non-null   object 
 8   tour_arrangement       4809 non-null   object 
 9   package_transport_int  4809 non-null   object 
 10  package_accomodation   4809 non-null   object 
 11  package_food           4809 non-null   object 
 12  package_transport_tz   4809 non-null   object 
 13  package_sightseeing    4809 non-null   object 
 14  package_guided_tour    4809 non-null   object 
 15  pack

In [10]:
two_categ = []
for col in df.columns:
    if len(df[col].value_counts()) <= 2:
        two_categ.append(col)


In [11]:
two_categ

['tour_arrangement',
 'package_transport_int',
 'package_accomodation',
 'package_food',
 'package_transport_tz',
 'package_sightseeing',
 'package_guided_tour',
 'package_insurance',
 'first_trip_tz']

In [12]:
more_categ = []
for col in df.columns:
    if len(df[col].value_counts() > 2):
        more_categ.append(col)

In [16]:
len(more_categ)

22

In [13]:
one_hot_cols = []
for col in more_categ:
    if df[col].dtypes == 'object':
        one_hot_cols.append(col)

In [15]:
len(one_hot_cols)

17

In [26]:
label_encode_cols = []
for col in two_categ:
    if df[col].dtypes == 'object':
        label_encode_cols.append(col)

In [24]:
len(label_encode_cols)

0

In [27]:
df = pd.get_dummies(df, columns=one_hot_cols)

In [28]:
df.head()

,total_female,total_male,night_mainland,night_zanzibar,total_cost,country_ALGERIA,country_ANGOLA,country_ARGENTINA,country_AUSTRALIA,country_AUSTRIA,country_BELGIUM,country_BERMUDA,country_BRAZIL,country_BURGARIA,country_BURUNDI,country_CANADA,country_CAPE VERDE,country_CHILE,country_CHINA,country_COLOMBIA,country_COMORO,country_CONGO,country_COSTARICA,country_CROATIA,country_CYPRUS,country_CZECH REPUBLIC,country_DENMARK,country_DJIBOUT,country_DOMINICA,country_DRC,country_EGYPT,country_ESTONIA,country_ETHIOPIA,country_FINLAND,country_FRANCE,country_GEORGIA,country_GERMANY,country_GHANA,country_GREECE,country_HUNGARY,country_INDIA,country_INDONESIA,country_IRAN,country_IRAQ,country_IRELAND,country_ISRAEL,country_ITALY,country_IVORY COAST,country_JAPAN,country_KENYA,country_KOREA,country_KUWAIT,country_LATVIA,country_LEBANON,country_LESOTHO,country_LITHUANIA,country_LUXEMBOURG,country_MADAGASCAR,country_MALAWI,country_MALAYSIA,country_MALT,country_MAURITIUS,country_MEXICO,country_MONTENEGRO,country_MORROCO,country_MOZAMBIQUE,country_MYANMAR,country_NAMIBIA,country_NEPAL,country_NETHERLANDS,country_NEW ZEALAND,country_NIGER,country_NIGERIA,country_NORWAY,country_OMAN,country_PAKISTAN,country_PHILIPINES,country_POLAND,country_PORTUGAL,country_QATAR,country_ROMANIA,country_RUSSIA,country_RWANDA,country_SCOTLAND,country_SERBIA,country_SINGAPORE,country_SLOVAKIA,country_SLOVENIA,country_SOUTH AFRICA,country_SPAIN,country_SRI LANKA,country_SUDAN,country_SWAZILAND,country_SWEDEN,country_SWIZERLAND,country_TAIWAN,country_THAILAND,country_TRINIDAD TOBACCO,country_TUNISIA,country_TURKEY,country_UAE,country_UGANDA,country_UKRAIN,country_UNITED ARAB EMIRATES,country_UNITED KINGDOM,country_UNITED STATES OF AMERICA,country_URUGUAY,country_YEMEN,country_ZAMBIA,country_ZIMBABWE,age_group_1-24,age_group_25-44,age_group_45-64,age_group_65+,travel_with_Alone,travel_with_Children,travel_with_Friends/Relatives,travel_with_Spouse,travel_with_Spouse and Children,purpose_Business,purpose_Leisure and Holidays,purpose_Meetings and Conference,purpose_Other,purpose_Scientific and Academic,purpose_Visiting Friends and Relatives,purpose_Volunteering,main_activity_Beach tourism,main_activity_Bird watching,main_activity_Conference tourism,main_activity_Cultural tourism,main_activity_Diving and Sport Fishing,main_activity_Hunting tourism,main_activity_Mountain climbing,main_activity_Wildlife tourism,main_activity_business,"info_source_Friends, relatives","info_source_Newspaper, magazines,brochures","info_source_Radio, TV, Web",info_source_Tanzania Mission Abroad,info_source_Trade fair,"info_source_Travel, agent, tour operator",info_source_inflight magazines,info_source_others,tour_arrangement_Independent,tour_arrangement_Package Tour,package_transport_int_No,package_transport_int_Yes,package_accomodation_No,package_accomodation_Yes,package_food_No,package_food_Yes,package_transport_tz_No,package_transport_tz_Yes,package_sightseeing_No,package_sightseeing_Yes,package_guided_tour_No,package_guided_tour_Yes,package_insurance_No,package_insurance_Yes,payment_mode_Cash,payment_mode_Credit Card,payment_mode_Other,payment_mode_Travellers Cheque,first_trip_tz_No,first_trip_tz_Yes,most_impressing_ Wildlife,most_impressing_Excellent Experience,most_impressing_Friendly People,most_impressing_Good service,most_impressing_No comments,most_impressing_Satisfies and Hope Come Back,"most_impressing_Wonderful Country, Landscape, Nature"
0,1.0,1.0,13.0,0.0,674602.5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,F

In [29]:
le = LabelEncoder()

for col in label_encode_cols:
    if col in df.columns:
        df[col] = le.fit_transform(df[col])

In [30]:
print(df.shape)
df.sample(5)

(4809, 172)


,total_female,total_male,night_mainland,night_zanzibar,total_cost,country_ALGERIA,country_ANGOLA,country_ARGENTINA,country_AUSTRALIA,country_AUSTRIA,country_BELGIUM,country_BERMUDA,country_BRAZIL,country_BURGARIA,country_BURUNDI,country_CANADA,country_CAPE VERDE,country_CHILE,country_CHINA,country_COLOMBIA,country_COMORO,country_CONGO,country_COSTARICA,country_CROATIA,country_CYPRUS,country_CZECH REPUBLIC,country_DENMARK,country_DJIBOUT,country_DOMINICA,country_DRC,country_EGYPT,country_ESTONIA,country_ETHIOPIA,country_FINLAND,country_FRANCE,country_GEORGIA,country_GERMANY,country_GHANA,country_GREECE,country_HUNGARY,country_INDIA,country_INDONESIA,country_IRAN,country_IRAQ,country_IRELAND,country_ISRAEL,country_ITALY,country_IVORY COAST,country_JAPAN,country_KENYA,country_KOREA,country_KUWAIT,country_LATVIA,country_LEBANON,country_LESOTHO,country_LITHUANIA,country_LUXEMBOURG,country_MADAGASCAR,country_MALAWI,country_MALAYSIA,country_MALT,country_MAURITIUS,country_MEXICO,country_MONTENEGRO,country_MORROCO,country_MOZAMBIQUE,country_MYANMAR,country_NAMIBIA,country_NEPAL,country_NETHERLANDS,country_NEW ZEALAND,country_NIGER,country_NIGERIA,country_NORWAY,country_OMAN,country_PAKISTAN,country_PHILIPINES,country_POLAND,country_PORTUGAL,country_QATAR,country_ROMANIA,country_RUSSIA,country_RWANDA,country_SCOTLAND,country_SERBIA,country_SINGAPORE,country_SLOVAKIA,country_SLOVENIA,country_SOUTH AFRICA,country_SPAIN,country_SRI LANKA,country_SUDAN,country_SWAZILAND,country_SWEDEN,country_SWIZERLAND,country_TAIWAN,country_THAILAND,country_TRINIDAD TOBACCO,country_TUNISIA,country_TURKEY,country_UAE,country_UGANDA,country_UKRAIN,country_UNITED ARAB EMIRATES,country_UNITED KINGDOM,country_UNITED STATES OF AMERICA,country_URUGUAY,country_YEMEN,country_ZAMBIA,country_ZIMBABWE,age_group_1-24,age_group_25-44,age_group_45-64,age_group_65+,travel_with_Alone,travel_with_Children,travel_with_Friends/Relatives,travel_with_Spouse,travel_with_Spouse and Children,purpose_Business,purpose_Leisure and Holidays,purpose_Meetings and Conference,purpose_Other,purpose_Scientific and Academic,purpose_Visiting Friends and Relatives,purpose_Volunteering,main_activity_Beach tourism,main_activity_Bird watching,main_activity_Conference tourism,main_activity_Cultural tourism,main_activity_Diving and Sport Fishing,main_activity_Hunting tourism,main_activity_Mountain climbing,main_activity_Wildlife tourism,main_activity_business,"info_source_Friends, relatives","info_source_Newspaper, magazines,brochures","info_source_Radio, TV, Web",info_source_Tanzania Mission Abroad,info_source_Trade fair,"info_source_Travel, agent, tour operator",info_source_inflight magazines,info_source_others,tour_arrangement_Independent,tour_arrangement_Package Tour,package_transport_int_No,package_transport_int_Yes,package_accomodation_No,package_accomodation_Yes,package_food_No,package_food_Yes,package_transport_tz_No,package_transport_tz_Yes,package_sightseeing_No,package_sightseeing_Yes,package_guided_tour_No,package_guided_tour_Yes,package_insurance_No,package_insurance_Yes,payment_mode_Cash,payment_mode_Credit Card,payment_mode_Other,payment_mode_Travellers Cheque,first_trip_tz_No,first_trip_tz_Yes,most_impressing_ Wildlife,most_impressing_Excellent Experience,most_impressing_Friendly People,most_impressing_Good service,most_impressing_No comments,most_impressing_Satisfies and Hope Come Back,"most_impressing_Wonderful Country, Landscape, Nature"
4372,0.0,1.0,2.0,0.0,1160250.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

In [31]:
df.fillna(0,inplace=True)
df = df.astype(int)

In [36]:
df

,total_female,total_male,night_mainland,night_zanzibar,total_cost,country_ALGERIA,country_ANGOLA,country_ARGENTINA,country_AUSTRALIA,country_AUSTRIA,country_BELGIUM,country_BERMUDA,country_BRAZIL,country_BURGARIA,country_BURUNDI,country_CANADA,country_CAPE VERDE,country_CHILE,country_CHINA,country_COLOMBIA,country_COMORO,country_CONGO,country_COSTARICA,country_CROATIA,country_CYPRUS,country_CZECH REPUBLIC,country_DENMARK,country_DJIBOUT,country_DOMINICA,country_DRC,country_EGYPT,country_ESTONIA,country_ETHIOPIA,country_FINLAND,country_FRANCE,country_GEORGIA,country_GERMANY,country_GHANA,country_GREECE,country_HUNGARY,country_INDIA,country_INDONESIA,country_IRAN,country_IRAQ,country_IRELAND,country_ISRAEL,country_ITALY,country_IVORY COAST,country_JAPAN,country_KENYA,country_KOREA,country_KUWAIT,country_LATVIA,country_LEBANON,country_LESOTHO,country_LITHUANIA,country_LUXEMBOURG,country_MADAGASCAR,country_MALAWI,country_MALAYSIA,country_MALT,country_MAURITIUS,country_MEXICO,country_MONTENEGRO,country_MORROCO,country_MOZAMBIQUE,country_MYANMAR,country_NAMIBIA,country_NEPAL,country_NETHERLANDS,country_NEW ZEALAND,country_NIGER,country_NIGERIA,country_NORWAY,country_OMAN,country_PAKISTAN,country_PHILIPINES,country_POLAND,country_PORTUGAL,country_QATAR,country_ROMANIA,country_RUSSIA,country_RWANDA,country_SCOTLAND,country_SERBIA,country_SINGAPORE,country_SLOVAKIA,country_SLOVENIA,country_SOUTH AFRICA,country_SPAIN,country_SRI LANKA,country_SUDAN,country_SWAZILAND,country_SWEDEN,country_SWIZERLAND,country_TAIWAN,country_THAILAND,country_TRINIDAD TOBACCO,country_TUNISIA,country_TURKEY,country_UAE,country_UGANDA,country_UKRAIN,country_UNITED ARAB EMIRATES,country_UNITED KINGDOM,country_UNITED STATES OF AMERICA,country_URUGUAY,country_YEMEN,country_ZAMBIA,country_ZIMBABWE,age_group_1-24,age_group_25-44,age_group_45-64,age_group_65+,travel_with_Alone,travel_with_Children,travel_with_Friends/Relatives,travel_with_Spouse,travel_with_Spouse and Children,purpose_Business,purpose_Leisure and Holidays,purpose_Meetings and Conference,purpose_Other,purpose_Scientific and Academic,purpose_Visiting Friends and Relatives,purpose_Volunteering,main_activity_Beach tourism,main_activity_Bird watching,main_activity_Conference tourism,main_activity_Cultural tourism,main_activity_Diving and Sport Fishing,main_activity_Hunting tourism,main_activity_Mountain climbing,main_activity_Wildlife tourism,main_activity_business,"info_source_Friends, relatives","info_source_Newspaper, magazines,brochures","info_source_Radio, TV, Web",info_source_Tanzania Mission Abroad,info_source_Trade fair,"info_source_Travel, agent, tour operator",info_source_inflight magazines,info_source_others,tour_arrangement_Independent,tour_arrangement_Package Tour,package_transport_int_No,package_transport_int_Yes,package_accomodation_No,package_accomodation_Yes,package_food_No,package_food_Yes,package_transport_tz_No,package_transport_tz_Yes,package_sightseeing_No,package_sightseeing_Yes,package_guided_tour_No,package_guided_tour_Yes,package_insurance_No,package_insurance_Yes,payment_mode_Cash,payment_mode_Credit Card,payment_mode_Other,payment_mode_Travellers Cheque,first_trip_tz_No,first_trip_tz_Yes,most_impressing_ Wildlife,most_impressing_Excellent Experience,most_impressing_Friendly People,most_impressing_Good service,most_impressing_No comments,most_impressing_Satisfies and Hope Come Back,"most_impressing_Wonderful Country, Landscape, Nature"
0,1,1,13,0,674602,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0
1,1,0,14,7,3214906,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [58]:
df_com = df[common_cols]

In [59]:
X = df_com
y = df['total_cost']

In [60]:
model = RandomForestRegressor()

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size = 0.2, random_state = 42)

In [62]:
model = model.fit(X_train, y_train)

In [63]:
model.score(X_test, y_test)

0.3974195043374097

In [64]:
model.score(X_train, y_train)

0.908087558623922

## Test dataset

In [43]:
df_test_o = pd.read_csv("Test.csv")

In [44]:
df_test_o['age_group'] = df_test_o['age_group'].replace({'24-Jan':'1-24'})

In [45]:
df_test = df_test_o.drop(columns = ['ID'],  axis=1)

In [352]:
df_test.head()

,country,age_group,travel_with,total_female,total_male,purpose,main_activity,info_source,tour_arrangement,package_transport_int,package_accomodation,package_food,package_transport_tz,package_sightseeing,package_guided_tour,package_insurance,night_mainland,night_zanzibar,payment_mode,first_trip_tz,most_impressing
0,AUSTRALIA,45-64,Spouse,1.0,1.0,Leisure and Holidays,Wildlife tourism,"Travel, agent, tour operator",Package Tour,Yes,Yes,Yes,Yes,Yes,Yes,Yes,10,3,Cash,Yes,Wildlife
1,SOUTH AFRICA,25-44,Friends/Relatives,0.0,4.0,Business,Wildlife tourism,Tanzania Mission Abroad,Package Tour,Yes,Yes,No,No,No,No,No,13,0,Cash,No,"Wonderful Country, Landscape, Nature"
2,GERMANY,25-44,Friends/Relatives,3.0,0.0,Leisure and Holidays,Beach tourism,"Friends, relatives",Independent,No,No,No,No,No,No,No,7,14,Cash,No,No comments
3,CANADA,1-24,Friends/Relatives,2.0,0.0,Leisure and Holidays,Cultural tourism,others,Independent,No,No,No,No,No,No,No,0,4,Cash,Yes,Friendly People
4,UNITED KINGDOM,45-64,Friends/Relatives,2.0,2.0,Leisure and Holidays,Wildlife tourism,"Friends, relatives",Package Tour,Yes,Yes,Yes,Yes,No,No,No,10,0,Cash,Yes,Friendly People


In [46]:
df_test = pd.get_dummies(df_test, columns=one_hot_cols)

In [47]:
for col in label_encode_cols:
    if col in df_test.columns:
        df_test[col] = le.fit_transform(df_test[col])

In [48]:
df_test.columns

Index(['total_female', 'total_male', 'night_mainland', 'night_zanzibar',
       'country_ARGENTINA', 'country_AUSTRALIA', 'country_AUSTRIA',
       'country_BANGLADESH', 'country_BELGIUM', 'country_BOSNIA',
       ...
       'payment_mode_Travellers Cheque', 'first_trip_tz_No',
       'first_trip_tz_Yes', 'most_impressing_ Wildlife',
       'most_impressing_Excellent Experience',
       'most_impressing_Friendly People', 'most_impressing_Good service',
       'most_impressing_No comments',
       'most_impressing_Satisfies and Hope Come Back',
       'most_impressing_Wonderful Country, Landscape, Nature'],
      dtype='object', length=153)

In [49]:
common_cols = [col for col in X.columns if col in df_test.columns]

In [50]:
df_test = df_test[common_cols]

In [51]:
df.columns

Index(['total_female', 'total_male', 'night_mainland', 'night_zanzibar',
       'total_cost', 'country_ALGERIA', 'country_ANGOLA', 'country_ARGENTINA',
       'country_AUSTRALIA', 'country_AUSTRIA',
       ...
       'payment_mode_Travellers Cheque', 'first_trip_tz_No',
       'first_trip_tz_Yes', 'most_impressing_ Wildlife',
       'most_impressing_Excellent Experience',
       'most_impressing_Friendly People', 'most_impressing_Good service',
       'most_impressing_No comments',
       'most_impressing_Satisfies and Hope Come Back',
       'most_impressing_Wonderful Country, Landscape, Nature'],
      dtype='object', length=172)

In [52]:
df_test.columns

Index(['total_female', 'total_male', 'night_mainland', 'night_zanzibar',
       'country_ARGENTINA', 'country_AUSTRALIA', 'country_AUSTRIA',
       'country_BELGIUM', 'country_BRAZIL', 'country_BURUNDI',
       ...
       'payment_mode_Travellers Cheque', 'first_trip_tz_No',
       'first_trip_tz_Yes', 'most_impressing_ Wildlife',
       'most_impressing_Excellent Experience',
       'most_impressing_Friendly People', 'most_impressing_Good service',
       'most_impressing_No comments',
       'most_impressing_Satisfies and Hope Come Back',
       'most_impressing_Wonderful Country, Landscape, Nature'],
      dtype='object', length=140)

In [53]:
df_test.fillna(0, inplace=True)
df_test = df_test.astype(int)

In [54]:
df_test.head()

,total_female,total_male,night_mainland,night_zanzibar,country_ARGENTINA,country_AUSTRALIA,country_AUSTRIA,country_BELGIUM,country_BRAZIL,country_BURUNDI,country_CANADA,country_CAPE VERDE,country_CHILE,country_CHINA,country_COMORO,country_CONGO,country_COSTARICA,country_CYPRUS,country_CZECH REPUBLIC,country_DENMARK,country_DOMINICA,country_DRC,country_EGYPT,country_ETHIOPIA,country_FINLAND,country_FRANCE,country_GERMANY,country_GHANA,country_GREECE,country_HUNGARY,country_INDIA,country_INDONESIA,country_IRAN,country_IRELAND,country_ISRAEL,country_ITALY,country_JAPAN,country_KENYA,country_KOREA,country_LEBANON,country_MALAWI,country_MALAYSIA,country_MALT,country_MORROCO,country_MOZAMBIQUE,country_NAMIBIA,country_NETHERLANDS,country_NEW ZEALAND,country_NORWAY,country_OMAN,country_PAKISTAN,country_PHILIPINES,country_POLAND,country_PORTUGAL,country_QATAR,country_ROMANIA,country_RUSSIA,country_RWANDA,country_SCOTLAND,country_SERBIA,country_SINGAPORE,country_SLOVAKIA,country_SOUTH AFRICA,country_SPAIN,country_SUDAN,country_SWAZILAND,country_SWEDEN,country_SWIZERLAND,country_TAIWAN,country_UAE,country_UGANDA,country_UKRAIN,country_UNITED ARAB EMIRATES,country_UNITED KINGDOM,country_UNITED STATES OF AMERICA,country_YEMEN,country_ZAMBIA,country_ZIMBABWE,age_group_1-24,age_group_25-44,age_group_45-64,age_group_65+,travel_with_Alone,travel_with_Children,travel_with_Friends/Relatives,travel_with_Spouse,travel_with_Spouse and Children,purpose_Business,purpose_Leisure and Holidays,purpose_Meetings and Conference,purpose_Other,purpose_Scientific and Academic,purpose_Visiting Friends and Relatives,purpose_Volunteering,main_activity_Beach tourism,main_activity_Bird watching,main_activity_Conference tourism,main_activity_Cultural tourism,main_activity_Diving and Sport Fishing,main_activity_Hunting tourism,main_activity_Mountain climbing,main_activity_Wildlife tourism,main_activity_business,"info_source_Friends, relatives","info_source_Newspaper, magazines,brochures","info_source_Radio, TV, Web",info_source_Tanzania Mission Abroad,info_source_Trade fair,"info_source_Travel, agent, tour operator",info_source_inflight magazines,info_source_others,tour_arrangement_Independent,tour_arrangement_Package Tour,package_transport_int_No,package_transport_int_Yes,package_accomodation_No,package_accomodation_Yes,package_food_No,package_food_Yes,package_transport_tz_No,package_transport_tz_Yes,package_sightseeing_No,package_sightseeing_Yes,package_guided_tour_No,package_guided_tour_Yes,package_insurance_No,package_insurance_Yes,payment_mode_Cash,payment_mode_Credit Card,payment_mode_Other,payment_mode_Travellers Cheque,first_trip_tz_No,first_trip_tz_Yes,most_impressing_ Wildlife,most_impressing_Excellent Experience,most_impressing_Friendly People,most_impressing_Good service,most_impressing_No comments,most_impressing_Satisfies and Hope Come Back,"most_impressing_Wonderful Country, Landscape, Nature"
0,1,1,10,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0
1,0,4,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1
2,3,0,7,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0
3,2,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1

In [55]:
df_test.shape

(1601, 140)

In [56]:
df_test_o.shape

(1601, 22)

In [65]:
preds = model.predict(df_test)

In [363]:
prediction = pd.DataFrame({'ID' : df_test_o['ID'],
                          'preds' : preds})

In [364]:
prediction.to_csv('prediction.csv', index=False)

In [365]:
prediction.sample(10)

,ID,preds
1154,tour_5216,1.540912e+07
131,tour_1441,7.214617e+05
1076,tour_4939,8.245154e+06
1,tour_100,1.443968e+07
123,tour_1421,4.519044e+06
1122,tour_5113,9.525626e+06
769,tour_3739,1.648044e+07
72,tour_125,3.144252e+06
1006,tour_4621,2.527651e+07
795,tour_384,2.957433e+06
